In [1]:
from datasets import dog_dataset, cub_dataset, food_dataset, cub_and_dogs
from models.models_to_finetune import deit_small_patch16_224, myresnetv2_task1, myresnetv2_task2, myresnetv2_for_c_loss
import PIL
import numpy as np
from tqdm import tqdm
import torch
import torch.optim as optim
from torchvision import transforms
import config
import sys
import math
from loss import CenterLoss
from run_center_loss import train_model_with_closs
from vit.vit_pytorch.nest import NesT
import timm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from models import bilinear_model
from run import train_model
from loss import GBLoss

All of the models can be trained using the two **main.py** files in the submission folder. These notebook contain sufficient code to run inference on the selected models for the sake of clarity.

In [2]:
epochs = 50
batch_size = 32
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
test_transform=transforms.Compose([
                    transforms.Resize((448, 448)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])

data_transform4 = transforms.Compose([  #

        transforms.Resize((448, 448)),
        transforms.RandomRotation(20),
        transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

*In task1 resnetv2-448 gave the best results for CNN-based model, while cait_xxs_24_384 gave the best results for transformer-based model on CUB-dataset. In this task we will try use these models as baseline and run different experiments by changing the loss functions as well as the architecture.* Training longer increased accuracy for each experiment setting; for a fair comparison we will compare accuracy achieved within 30 epochs. \
**Kindly check the excel sheet provided in the submission to look through all the experiments done for task 2.**

## Cait_xxs_24_384 
Batch size = 24 during training

In [4]:
test_transform=transforms.Compose([
                    transforms.Resize((384, 384)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])

data_transform = transforms.Compose([  #

       transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

# Load Data
train_loader, val_loader,test_loader = cub_and_dogs(bs=batch_size, data_transform=data_transform, test_transform=test_transform)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = timm.create_model("cait_xxs24_384", pretrained=True)
model.head = torch.nn.Linear(in_features=model.head.in_features, out_features=320)  # dogs dataset has 120 classes
model.head.apply(model._init_weights)
model.to(device)
path = "/home/hashmat.malik/Fall 2021/CV703 Lab/Week5/datasets/modelcait_xxs24_384_task2exp7_for_nb_best.pth.tar"
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['state_dict'])
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.5, 0.999))

In [5]:
train_model(epochs, train_loader, val_loader, test_loader, optimizer, criterion, model, 'cait', is_train=False)

Test: [ 0/57]	Time  1.164 ( 1.164)	Loss 4.2061e-01 (4.2061e-01)	Acc@1  84.38 ( 84.38)	Acc@5 100.00 (100.00)
Test: [ 5/57]	Time  0.346 ( 0.491)	Loss 9.1943e-01 (6.3828e-01)	Acc@1  71.88 ( 82.29)	Acc@5  96.88 ( 96.35)
Test: [10/57]	Time  0.344 ( 0.426)	Loss 5.3010e-01 (6.9046e-01)	Acc@1  90.62 ( 82.10)	Acc@5  96.88 ( 96.02)
Test: [15/57]	Time  0.383 ( 0.405)	Loss 4.4733e-01 (6.5309e-01)	Acc@1  87.50 ( 83.01)	Acc@5 100.00 ( 96.88)
Test: [20/57]	Time  0.384 ( 0.400)	Loss 7.2690e-01 (6.5689e-01)	Acc@1  84.38 ( 83.04)	Acc@5  93.75 ( 97.02)
Test: [25/57]	Time  0.345 ( 0.394)	Loss 8.8095e-01 (6.5660e-01)	Acc@1  84.38 ( 82.93)	Acc@5  93.75 ( 96.88)
Test: [30/57]	Time  0.384 ( 0.391)	Loss 7.4438e-01 (6.5257e-01)	Acc@1  84.38 ( 83.27)	Acc@5  93.75 ( 96.67)
Test: [35/57]	Time  0.387 ( 0.390)	Loss 6.2901e-01 (6.5474e-01)	Acc@1  81.25 ( 82.81)	Acc@5 100.00 ( 96.79)
Test: [40/57]	Time  0.347 ( 0.389)	Loss 3.8881e-01 (6.3285e-01)	Acc@1  90.62 ( 83.31)	Acc@5 100.00 ( 96.95)
Test: [45/57]	Time  0.346 ( 

Test: [315/450]	Time  0.384 ( 0.379)	Loss 6.4210e-01 (5.2206e-01)	Acc@1  90.62 ( 85.28)	Acc@5  93.75 ( 98.45)
Test: [320/450]	Time  0.391 ( 0.380)	Loss 1.3081e+00 (5.2988e-01)	Acc@1  65.62 ( 85.11)	Acc@5  93.75 ( 98.41)
Test: [325/450]	Time  0.389 ( 0.379)	Loss 1.3119e+00 (5.4028e-01)	Acc@1  68.75 ( 84.79)	Acc@5  96.88 ( 98.32)
Test: [330/450]	Time  0.354 ( 0.379)	Loss 1.7280e+00 (5.4660e-01)	Acc@1  56.25 ( 84.61)	Acc@5  81.25 ( 98.29)
Test: [335/450]	Time  0.379 ( 0.379)	Loss 5.8474e-01 (5.4679e-01)	Acc@1  90.62 ( 84.67)	Acc@5 100.00 ( 98.29)
Test: [340/450]	Time  0.354 ( 0.379)	Loss 1.2122e+00 (5.5225e-01)	Acc@1  62.50 ( 84.59)	Acc@5  93.75 ( 98.23)
Test: [345/450]	Time  0.354 ( 0.379)	Loss 7.0318e-01 (5.5431e-01)	Acc@1  81.25 ( 84.63)	Acc@5  93.75 ( 98.21)
Test: [350/450]	Time  0.353 ( 0.379)	Loss 5.1256e-01 (5.6059e-01)	Acc@1  96.88 ( 84.60)	Acc@5 100.00 ( 98.11)
Test: [355/450]	Time  0.355 ( 0.379)	Loss 1.1747e+00 (5.6739e-01)	Acc@1  87.50 ( 84.55)	Acc@5  90.62 ( 98.02)
Test: [360

Reached top1 validation accuracy of 82.667% and top1 test accuracy of 82.587%. However, when we keep the same batch size (i.e same number of updates) for both resnetv2 and cait, cait's performance decreases. Please check the submitted for excel sheet for detailed results.

## Resnetv2-448
Batch size = 96 during training

In [5]:
epochs = 50
batch_size = 24
import torch.nn as nn

test_transform=transforms.Compose([
                    transforms.Resize((448, 448)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])

transform = transforms.Compose([ 

        transforms.CenterCrop(384),
        transforms.RandomRotation(20),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

# Load Data
train_loader, val_loader,test_loader = cub_and_dogs(bs=batch_size, data_transform = transform, test_transform=test_transform)
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

model = myresnetv2_task2(num_classes=320)

model = model.to(device)

path = "/home/hashmat.malik/Fall 2021/CV703 Lab/Week5/datasets/modelresnetv2448_submission_task2_exp2_best.pth.tar"
checkpoint = torch.load(path)

model.load_state_dict(checkpoint['state_dict'])



optimizer = model.parameters([
                {'params': 0.99, 'lr': 0.0001, 'betas': (0.5, 0.999)},
                {'params': 0.99, 'lr': 0.00001, 'betas': (0.5, 0.999)}])

criterion = torch.nn.CrossEntropyLoss()
train_model(30, train_loader, val_loader, test_loader, optimizer, criterion, model, f'resnetv2_fusion_{1}', is_train=False)

Test: [ 0/75]	Time  0.704 ( 0.704)	Loss 6.3873e-01 (6.3873e-01)	Acc@1  87.50 ( 87.50)	Acc@5 100.00 (100.00)
Test: [ 5/75]	Time  0.508 ( 0.553)	Loss 1.0646e+00 (8.0182e-01)	Acc@1  66.67 ( 78.47)	Acc@5  91.67 ( 96.53)
Test: [10/75]	Time  0.517 ( 0.536)	Loss 6.9550e-01 (7.5518e-01)	Acc@1  79.17 ( 76.89)	Acc@5 100.00 ( 96.59)
Test: [15/75]	Time  0.493 ( 0.527)	Loss 4.8774e-01 (7.2447e-01)	Acc@1  87.50 ( 77.60)	Acc@5 100.00 ( 97.14)
Test: [20/75]	Time  0.503 ( 0.522)	Loss 6.4719e-01 (6.9613e-01)	Acc@1  83.33 ( 78.57)	Acc@5  91.67 ( 97.02)
Test: [25/75]	Time  0.554 ( 0.523)	Loss 6.5845e-01 (6.6821e-01)	Acc@1  75.00 ( 79.49)	Acc@5  95.83 ( 96.96)
Test: [30/75]	Time  0.510 ( 0.520)	Loss 7.1005e-01 (6.4506e-01)	Acc@1  83.33 ( 80.11)	Acc@5 100.00 ( 97.45)
Test: [35/75]	Time  0.483 ( 0.519)	Loss 3.8224e-01 (6.2733e-01)	Acc@1  87.50 ( 81.48)	Acc@5 100.00 ( 97.34)
Test: [40/75]	Time  0.496 ( 0.518)	Loss 4.1304e-01 (6.3160e-01)	Acc@1  87.50 ( 81.50)	Acc@5 100.00 ( 97.36)
Test: [45/75]	Time  0.484 ( 

Reached top1 validation accuracy of 82.278% and top1 test accuracy of 82.816%

## Center Loss
In addition to cross entropy loss used for classification, we also tried to incorporate center loss into the objective function. The reasoning behind this was as there is less inter-class variation among the classes, center-loss would try to separate the classes in the feature space for easier classification. **The implementation of the loss function is in loss.py file.**
Here we are showing the results of center loss on resnetv2-448 using cub and dogs dataset. For comparison , we are only showing models trained without any data augmentation (just resize and normalisation) for the sake of clarity. Best Top1 val accuracy on resnetv2-448 with cross-entropy loss was 82.28 and best top1 test accuracy with cross entropy loss was 82.816%

In [15]:
# implementation of dataloaders is done in dataset.py file.
data_transform = transforms.Compose([  #

        transforms.Resize((448, 448)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])
train_loader, val_loader, test_loader = cub_and_dogs(bs=batch_size, data_transform=data_transform, test_transform=test_transform)

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = myresnetv2_for_c_loss(num_classes=320)
model = model.to(device)

for param in model.parameters():
    param.requires_grad = True

my_list = ['head.1.weight', 'head.1.bias','head.3.weight', 'head.3.bias']
params = list(filter(lambda kv: kv[0] in my_list, model.named_parameters()))
base_params = list(filter(lambda kv: kv[0] not in my_list, model.named_parameters()))

crit_entr = torch.nn.CrossEntropyLoss()
crit_closs = CenterLoss(num_classes=320, feat_dim=512)
path = "/home/hashmat.malik/Fall 2021/CV703 Lab/Week5/datasets/Task2:cub_and_dogs/Exp1/model_best_resnet_v2_cubs_dogs_0.pth.tar"
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['state_dict'])
optimizer = optim.Adam([{'params':  [i[1]for i in params], 'lr': 0.0001, 'betas': (0.5, 0.999)},
                {'params':  [i[1]for i in base_params], 'lr': 0.00001, 'betas': (0.5, 0.999)},
                {'params': crit_closs.parameters(), 'lr': 0.01, 'betas': (0.5, 0.999)}
                        ])

scheduler = ReduceLROnPlateau(optimizer, 'max')
train_model_with_closs(30, train_loader, val_loader, test_loader, optimizer, scheduler, crit_entr, crit_closs, model, f'resnet_v2_closs_new_lr_{0.01}', is_train=False)


Test: [ 0/57]	Time  0.670 ( 0.670)	ent_Loss 4.6718e-01 (4.6718e-01)	center_loss 2.7889e+03 (2.7889e+03)	loss 8.8340e+00 (8.8340e+00)	Acc@1  90.62 ( 90.62)	Acc@5  96.88 ( 96.88)
Test: [ 5/57]	Time  0.307 ( 0.392)	ent_Loss 6.0692e-01 (5.8485e-01)	center_loss 2.6108e+03 (2.6260e+03)	loss 8.4393e+00 (8.4629e+00)	Acc@1  87.50 ( 85.94)	Acc@5  96.88 ( 96.35)
Test: [10/57]	Time  0.344 ( 0.371)	ent_Loss 8.7803e-01 (6.5627e-01)	center_loss 2.6980e+03 (2.6443e+03)	loss 8.9719e+00 (8.5892e+00)	Acc@1  71.88 ( 83.81)	Acc@5 100.00 ( 97.16)
Test: [15/57]	Time  0.351 ( 0.369)	ent_Loss 5.8840e-01 (6.5883e-01)	center_loss 2.7748e+03 (2.6576e+03)	loss 8.9129e+00 (8.6316e+00)	Acc@1  84.38 ( 83.01)	Acc@5  93.75 ( 97.27)
Test: [20/57]	Time  0.341 ( 0.362)	ent_Loss 7.4801e-01 (6.5629e-01)	center_loss 2.5609e+03 (2.6474e+03)	loss 8.4308e+00 (8.5986e+00)	Acc@1  68.75 ( 82.59)	Acc@5  96.88 ( 97.32)
Test: [25/57]	Time  0.318 ( 0.355)	ent_Loss 7.6398e-01 (6.8024e-01)	center_loss 2.5907e+03 (2.6607e+03)	loss 8.5360

Reached top1 validation accuracy of $82.06\%$ and top1 test accuracy of $80.562\%$. This is still less than test accuracy of resnetv2-448 (82.09%) only trained using the cross-entropy loss. 

## Gradient-Boost Loss

In [6]:
# resnetv2-448
# batch_size = 48 used during training

In [4]:
test_transform=transforms.Compose([
                    transforms.Resize((448, 448)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])
data_transform = transforms.Compose([  #

        transforms.Resize((448, 448)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])
train_loader, val_loader, test_loader = cub_and_dogs(bs=batch_size, data_transform=data_transform, test_transform=test_transform)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

model = myresnetv2_task2(num_classes=320)

model = model.to(device)

path = "/home/hashmat.malik/Fall 2021/CV703 Lab/Week5/datasets/modelresnetv2448_gbloss_for_nb_best.pth.tar"
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['state_dict'])



optimizer = model.parameters([
                {'params': 0.99, 'lr': 0.0001, 'betas': (0.5, 0.999)},
                {'params': 0.99, 'lr': 0.00001, 'betas': (0.5, 0.999)}])

criterion = GBLoss()
train_model(30, train_loader, val_loader, test_loader, optimizer, criterion, model, f'resnetv2_gbloss_{1}', is_train=False)

Test: [ 0/57]	Time  0.989 ( 0.989)	Loss 6.3179e-01 (6.3179e-01)	Acc@1  87.50 ( 87.50)	Acc@5 100.00 (100.00)
Test: [ 5/57]	Time  0.328 ( 0.393)	Loss 6.5120e-01 (5.3780e-01)	Acc@1  78.12 ( 85.94)	Acc@5 100.00 ( 97.92)
Test: [10/57]	Time  0.365 ( 0.366)	Loss 4.1819e-01 (5.5087e-01)	Acc@1  87.50 ( 84.94)	Acc@5  96.88 ( 97.73)
Test: [15/57]	Time  0.320 ( 0.351)	Loss 5.7923e-01 (5.5709e-01)	Acc@1  84.38 ( 85.35)	Acc@5 100.00 ( 97.85)
Test: [20/57]	Time  0.395 ( 0.348)	Loss 3.7972e-01 (5.2617e-01)	Acc@1  90.62 ( 86.16)	Acc@5  96.88 ( 97.32)
Test: [25/57]	Time  0.329 ( 0.349)	Loss 6.2498e-01 (5.2313e-01)	Acc@1  87.50 ( 86.30)	Acc@5  96.88 ( 97.72)
Test: [30/57]	Time  0.377 ( 0.349)	Loss 7.4364e-01 (5.3379e-01)	Acc@1  84.38 ( 85.89)	Acc@5  90.62 ( 97.48)
Test: [35/57]	Time  0.326 ( 0.348)	Loss 2.9191e-01 (5.3602e-01)	Acc@1  93.75 ( 85.50)	Acc@5 100.00 ( 97.66)
Test: [40/57]	Time  0.308 ( 0.345)	Loss 6.2744e-01 (5.4051e-01)	Acc@1  81.25 ( 84.98)	Acc@5  96.88 ( 97.71)
Test: [45/57]	Time  0.309 ( 

**Best Top1 val accuracy of resnetv2-448 increased with gradient-boosting loss from  82.28% 84.17% to and best top1 test accuracy increased with gradient-boosting loss from 82.81% to 83.83%!!!** 

In [7]:
#cait_xxs24_384
# batch size = 24 used during training

In [8]:
test_transform=transforms.Compose([
                    transforms.Resize((384, 384)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])

data_transform = transforms.Compose([  #

       transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

# Load Data
train_loader, val_loader,test_loader = cub_and_dogs(bs=batch_size, data_transform=data_transform, test_transform=test_transform)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = timm.create_model("cait_xxs24_384", pretrained=True)
model.head = torch.nn.Linear(in_features=model.head.in_features, out_features=320)  # dogs dataset has 120 classes
model.head.apply(model._init_weights)
model.to(device)
path = "/home/hashmat.malik/Fall 2021/CV703 Lab/Week5/datasets/modelcait_xxs24_384_task2exp7withGBloss_for_nb_best.pth.tar"
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['state_dict'])
criterion = GBLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.5, 0.999))

In [9]:
train_model(epochs, train_loader, val_loader, test_loader, optimizer, criterion, model, 'cait', is_train=False)

Test: [ 0/57]	Time  0.786 ( 0.786)	Loss 3.3675e-01 (3.3675e-01)	Acc@1  90.62 ( 90.62)	Acc@5 100.00 (100.00)
Test: [ 5/57]	Time  0.344 ( 0.427)	Loss 8.1966e-01 (5.6247e-01)	Acc@1  68.75 ( 84.38)	Acc@5  96.88 ( 96.88)
Test: [10/57]	Time  0.343 ( 0.391)	Loss 4.3987e-01 (6.0817e-01)	Acc@1  87.50 ( 82.95)	Acc@5 100.00 ( 97.16)
Test: [15/57]	Time  0.352 ( 0.384)	Loss 3.8316e-01 (5.8015e-01)	Acc@1  87.50 ( 84.18)	Acc@5 100.00 ( 97.85)
Test: [20/57]	Time  0.347 ( 0.377)	Loss 5.6676e-01 (5.7327e-01)	Acc@1  81.25 ( 83.78)	Acc@5  93.75 ( 97.77)
Test: [25/57]	Time  0.382 ( 0.374)	Loss 7.0334e-01 (5.7251e-01)	Acc@1  75.00 ( 83.41)	Acc@5  93.75 ( 97.48)
Test: [30/57]	Time  0.378 ( 0.372)	Loss 6.1085e-01 (5.6795e-01)	Acc@1  81.25 ( 83.67)	Acc@5  93.75 ( 97.28)
Test: [35/57]	Time  0.367 ( 0.370)	Loss 4.9316e-01 (5.6904e-01)	Acc@1  84.38 ( 83.25)	Acc@5 100.00 ( 97.57)
Test: [40/57]	Time  0.386 ( 0.372)	Loss 2.9173e-01 (5.4908e-01)	Acc@1  96.88 ( 84.07)	Acc@5 100.00 ( 97.64)
Test: [45/57]	Time  0.385 ( 

Test: [315/450]	Time  0.399 ( 0.384)	Loss 4.4801e-01 (4.7014e-01)	Acc@1  90.62 ( 86.50)	Acc@5  96.88 ( 98.44)
Test: [320/450]	Time  0.399 ( 0.384)	Loss 1.3555e+00 (4.7764e-01)	Acc@1  53.12 ( 86.23)	Acc@5  96.88 ( 98.41)
Test: [325/450]	Time  0.403 ( 0.384)	Loss 1.1880e+00 (4.8745e-01)	Acc@1  75.00 ( 85.97)	Acc@5  96.88 ( 98.32)
Test: [330/450]	Time  0.399 ( 0.384)	Loss 1.9772e+00 (4.9537e-01)	Acc@1  50.00 ( 85.72)	Acc@5  75.00 ( 98.27)
Test: [335/450]	Time  0.398 ( 0.385)	Loss 4.4967e-01 (4.9560e-01)	Acc@1  90.62 ( 85.75)	Acc@5 100.00 ( 98.25)
Test: [340/450]	Time  0.399 ( 0.385)	Loss 1.1825e+00 (5.0087e-01)	Acc@1  56.25 ( 85.67)	Acc@5  90.62 ( 98.20)
Test: [345/450]	Time  0.399 ( 0.385)	Loss 5.5134e-01 (5.0258e-01)	Acc@1  90.62 ( 85.68)	Acc@5  93.75 ( 98.18)
Test: [350/450]	Time  0.399 ( 0.385)	Loss 4.8815e-01 (5.0632e-01)	Acc@1  93.75 ( 85.74)	Acc@5 100.00 ( 98.13)
Test: [355/450]	Time  0.399 ( 0.385)	Loss 1.1177e+00 (5.1289e-01)	Acc@1  84.38 ( 85.69)	Acc@5  87.50 ( 98.03)
Test: [360

**Best Top1 val accuracy of cait_xxs24_384 increased with gradient-boosting loss from  82.67%  to 83.83% and best top1 test accuracy increased with gradient-boosting loss from 82.59% to 84.07%!!!** 

**So  gradient-boosting loss improved the accuracy of both baseline models.Number of negative classes(k) to take into consideration is a hyperparameter for this Loss. We experimented with different values and found k=25 works best on this dataset.**

Now we will try to work on a hybrid model of resnetv2 and cait_xxs_24_384. We will pass image size of 384x384 to the hybrid model. Before using the hybrid model , let's check the individual accuracy on resnetv2-384 and cait_xxs_24_384.

## Resnetv2-384

In [11]:
epochs = 50
batch_size = 256
test_transform=transforms.Compose([
                    transforms.Resize((384, 384)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])
transform = transforms.Compose([ 
        transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])
# Load Data
train_loader, val_loader,test_loader = cub_and_dogs(bs=batch_size, data_transform = transform, test_transform=test_transform)
# This one uses a Multi GPU (nn.DataParallel) for training, thus for testing too
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

model = myresnetv2_task2(num_classes=320)


model = torch.nn.DataParallel(model)
model = model.to(device)

path = "/home/ikboljonsobirov/cv/ass1/Week5/datasets/model_resnetv2_cubs_and_dogs_noaug_best.pth.tar"
checkpoint = torch.load(path, map_location='cpu')

model.load_state_dict(checkpoint['state_dict'])

optimizer = model.parameters([
                {'params': 0.99, 'lr': 0.0001, 'betas': (0.5, 0.999)},
                {'params': 0.99, 'lr': 0.00001, 'betas': (0.5, 0.999)}])

criterion = torch.nn.CrossEntropyLoss()
train_model(30, train_loader, val_loader, test_loader, optimizer, criterion, model, f'resnetv2_fusion_{1}', is_train=False)

Test: [0/8]	Time  2.172 ( 2.172)	Loss 6.3603e-01 (6.3603e-01)	Acc@1  83.20 ( 83.20)	Acc@5  98.44 ( 98.44)
Test: [5/8]	Time  1.260 ( 1.371)	Loss 6.1622e-01 (6.5223e-01)	Acc@1  83.59 ( 82.23)	Acc@5  97.66 ( 97.59)
 * Acc@1 82.167 Acc@5 97.444
Test: [ 0/57]	Time  1.799 ( 1.799)	Loss 6.5389e-01 (6.5389e-01)	Acc@1  83.98 ( 83.98)	Acc@5  95.70 ( 95.70)
Test: [ 5/57]	Time  1.222 ( 1.325)	Loss 8.7307e-01 (6.8449e-01)	Acc@1  73.83 ( 81.18)	Acc@5  98.05 ( 97.07)
Test: [10/57]	Time  1.207 ( 1.285)	Loss 7.5338e-01 (6.8190e-01)	Acc@1  83.98 ( 82.03)	Acc@5  96.48 ( 97.12)
Test: [15/57]	Time  1.234 ( 1.261)	Loss 1.1194e+00 (7.3886e-01)	Acc@1  73.05 ( 80.64)	Acc@5  95.31 ( 97.00)
Test: [20/57]	Time  1.331 ( 1.259)	Loss 5.6061e-01 (6.7746e-01)	Acc@1  85.55 ( 81.85)	Acc@5  98.05 ( 97.36)
Test: [25/57]	Time  1.273 ( 1.262)	Loss 7.1457e-01 (6.7772e-01)	Acc@1  80.86 ( 81.64)	Acc@5  95.70 ( 97.31)
Test: [30/57]	Time  1.235 ( 1.260)	Loss 1.8646e-01 (6.5292e-01)	Acc@1  94.14 ( 81.99)	Acc@5  99.61 ( 97.54)
Tes

Reached top1 validation accuracy of 82.167% and top1 test accuracy of 81.613%

## Cait_xxs_24_384

In [13]:
test_transform=transforms.Compose([
                    transforms.Resize((384, 384)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])

data_transform = transforms.Compose([  #

       transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

# Load Data
train_loader, val_loader,test_loader = cub_and_dogs(bs=batch_size, data_transform=data_transform, test_transform=test_transform)
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = timm.create_model("cait_xxs24_384", pretrained=True)
model.head = torch.nn.Linear(in_features=model.head.in_features, out_features=320)  # dogs dataset has 120 classes

model = torch.nn.DataParallel(model)
model = model.to(device)


path = "/home/hashmat.malik/Fall 2021/CV703 Lab/Week5/datasets/Submission/Task2/Exp7/model_cait_cubs_and_dogs_noaug_best.pth.tar"
checkpoint = torch.load(path, map_location='cpu')
model.load_state_dict(checkpoint['state_dict'])
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.5, 0.999))

In [14]:
train_model(30, train_loader, val_loader, test_loader, optimizer, criterion, model, f'resnetv2_fusion_{1}', is_train=False)

Test: [ 0/57]	Time  0.861 ( 0.861)	Loss 3.6663e+00 (3.6663e+00)	Acc@1  43.75 ( 43.75)	Acc@5  59.38 ( 59.38)
Test: [ 5/57]	Time  0.348 ( 0.442)	Loss 3.0913e+00 (3.3674e+00)	Acc@1  59.38 ( 56.25)	Acc@5  68.75 ( 64.58)
Test: [10/57]	Time  0.348 ( 0.401)	Loss 4.7401e+00 (3.3411e+00)	Acc@1  37.50 ( 56.25)	Acc@5  46.88 ( 64.20)
Test: [15/57]	Time  0.349 ( 0.386)	Loss 2.8415e+00 (3.3730e+00)	Acc@1  56.25 ( 55.47)	Acc@5  71.88 ( 63.48)
Test: [20/57]	Time  0.348 ( 0.379)	Loss 3.2785e+00 (3.2950e+00)	Acc@1  50.00 ( 56.55)	Acc@5  65.62 ( 64.58)
Test: [25/57]	Time  0.350 ( 0.376)	Loss 4.3219e+00 (3.3283e+00)	Acc@1  40.62 ( 55.53)	Acc@5  56.25 ( 64.42)
Test: [30/57]	Time  0.392 ( 0.374)	Loss 3.2525e+00 (3.3463e+00)	Acc@1  59.38 ( 55.65)	Acc@5  62.50 ( 64.31)
Test: [35/57]	Time  0.395 ( 0.377)	Loss 2.3772e+00 (3.3527e+00)	Acc@1  71.88 ( 55.30)	Acc@5  75.00 ( 64.24)
Test: [40/57]	Time  0.358 ( 0.379)	Loss 5.3952e+00 (3.2942e+00)	Acc@1  37.50 ( 56.17)	Acc@5  43.75 ( 65.09)
Test: [45/57]	Time  0.393 ( 

In [ ]:


model = torch.nn.DataParallel(model)
model = model.to(device)

path = "/home/ikboljonsobirov/cv/ass1/Week5/datasets/model_resnetv2_cubs_and_dogs_noaug_best.pth.tar"
checkpoint = torch.load(path, map_location='cpu')

model.load_state_dict(checkpoint['state_dict'])

optimizer = model.parameters([
                {'params': 0.99, 'lr': 0.0001, 'betas': (0.5, 0.999)},
                {'params': 0.99, 'lr': 0.00001, 'betas': (0.5, 0.999)}])

criterion = torch.nn.CrossEntropyLoss()
train_model(30, train_loader, val_loader, test_loader, optimizer, criterion, model, f'resnetv2_fusion_{1}', is_train=False)

## Fusion Model
In this experiment we tried to combine the trasformer model (in our case cait_xxs_24_384) with a CNN model (in our case resnetv2). **The implementation of the model is in the models folder (bilinear_model.py)**. We extract features from the the third block of resnet backbone. For transformers, we combine all the patch token coming out of the encoder to form a feature map. Then we combine both of the feature maps using a transfusion module block in hope that it will be able to integrate the learned features from both of the backbones and give a more powerful feature representation.

For comparison , we are only showing models trained without any data augmentation (just resize and normalisation) for the sake of clarity. Training on larger resolution images gives a boost to the accuracy, as can be seen in using resnetv2 with $448\times 448$ image size. However to reduce computation and for a fair comparison with the Fusion model which uses $384 \times 384$ for both backbones, we will compare it with the accuracy of resnetv2_384 (top1 test Accuracy $81.61 \%$) and cait_384 (top1 test Accuracy $79.89 \%$).

In [8]:
test_transform=transforms.Compose([
                    transforms.Resize((384, 384)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])

data_transform = transforms.Compose([ 

        transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

train_loader, val_loader, test_loader = cub_and_dogs(bs=batch_size, data_transform=data_transform, test_transform=test_transform)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Here both backbones on pretrained on ImageNet. The bacbones are frozen, only the transfusion block and final layers are trained.

In [10]:
model = bilinear_model.TransFuse_S(num_classes=320, pretrained=True)
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
path = "/home/hashmat.malik/Fall 2021/CV703 Lab/Week5/datasets/modelresnetv2_fusion_4_best.pth.tar"
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['state_dict'])
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.5, 0.999))


In [14]:
train_model(30, train_loader, val_loader, test_loader, optimizer, criterion, model, f'resnetv2_fusion_{1}', is_train=False)

Test: [ 0/57]	Time  1.020 ( 1.020)	Loss 7.4345e-01 (7.4345e-01)	Acc@1  75.00 ( 75.00)	Acc@5  96.88 ( 96.88)
Test: [ 5/57]	Time  0.452 ( 0.559)	Loss 5.1658e-01 (6.4955e-01)	Acc@1  81.25 ( 78.12)	Acc@5 100.00 ( 97.92)
Test: [10/57]	Time  0.452 ( 0.513)	Loss 8.8987e-01 (7.5851e-01)	Acc@1  71.88 ( 76.42)	Acc@5 100.00 ( 97.44)
Test: [15/57]	Time  0.451 ( 0.496)	Loss 9.9967e-01 (8.0272e-01)	Acc@1  68.75 ( 75.98)	Acc@5  90.62 ( 96.48)
Test: [20/57]	Time  0.454 ( 0.487)	Loss 6.0313e-01 (7.6953e-01)	Acc@1  84.38 ( 77.23)	Acc@5  93.75 ( 96.28)
Test: [25/57]	Time  0.453 ( 0.481)	Loss 5.9471e-01 (7.4037e-01)	Acc@1  75.00 ( 78.12)	Acc@5 100.00 ( 96.63)
Test: [30/57]	Time  0.454 ( 0.478)	Loss 1.1169e+00 (7.4522e-01)	Acc@1  71.88 ( 77.92)	Acc@5  96.88 ( 96.67)
Test: [35/57]	Time  0.455 ( 0.476)	Loss 8.5409e-01 (7.1897e-01)	Acc@1  75.00 ( 78.99)	Acc@5  93.75 ( 96.88)
Test: [40/57]	Time  0.453 ( 0.474)	Loss 9.0299e-01 (7.1328e-01)	Acc@1  71.88 ( 78.81)	Acc@5 100.00 ( 97.18)
Test: [45/57]	Time  0.455 ( 

Reached top1 validation accuracy of $78.28\%$ and top1 test accuracy of $77.153\%$. This is less than the individual accuracy of both resnetv2-384 as well as cait_384. We also tried to fine tune the whole by unfreezing the backbones. However, it led to decrease in the accuracy which we believe is because loss of pretrained learned representation. Results can be seen in the excel sheet. While data augmentation increased the accuracy, it still was less than the accuracy achieved by resnetv2 and cait separately.